# Point Predictor Integration

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import patsy
import re
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.cross_validation import cross_val_score, StratifiedKFold, cross_val_predict
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV
import statsmodels.api as sm
import datetime as dt
from sklearn.ensemble import RandomForestRegressor
from scrapy.selector import Selector
#from scrapy import HtmlXPathSelector
from scrapy.http import HtmlResponse

%matplotlib inline
plt.style.use('fivethirtyeight')

In [14]:
def turn_to_float(val):
    #print val
    fl = float(val)
    return fl

def turn_to_int(val):
    #print val
    try:
        i = int(val)
    except:
        return val
    return i

def format_batter_numeric_rows(df):
    
    h = df.columns
    
    for i in np.arange(1, len(h), 1):
        
        try:
            df.loc[:, h[i]] = df[h[i]].map(turn_to_float)
        except:
            #print h[i]
            df.loc[:, h[i]] = df[h[i]]
            
    return df


In [15]:
def get_log_data(gamelogs_url, year, player):
    
    #url = 'http://www.baseball-reference.com/players/gl.cgi?id=crawfbr01&t=b&year=2015'
    url = gamelogs_url
    #print url
    r = requests.get(url)
    #    print "got to site"
    b = BeautifulSoup(r.text, 'html.parser')
    HTML = r.text 

    #Initialize row identifier
    RK = 1
    end_of_table = False
    
    #print gamelogs_url
    
    headers = [''.join(td.xpath('.//text()').extract()) for td in Selector(text=HTML).xpath('//div[3]/table/thead/tr/th')]
    
    #print len(headers)
    
    if len(headers) == 50:
        #It's a pitcher!
        log_data = pd.DataFrame(columns=headers)
        #print log_data.columns
        
    else:

        #Declare a new instance of a Dataframe log
        #It's a batter!
        log_data = pd.DataFrame(columns=headers)
#         log_data = pd.DataFrame(columns=(
#             'RK', 'Gcar', 'Gtm', 'Date', 'Tm', 'HoA', 'Opp', 'Rslt',
#            'Inngs', 'PA', 'AB', 'R', 'H', 'Doubles', 'Tripples', 'HR',
#            'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'ROE', 'GDP',
#            'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS', 'BOP', 'aLI', 'WPA',
#            'RE24', 'DK', 'FD', 'Pos')
#         )
    
    exception_cases = []

    #While RK is not null (i.e at the end of the table...)
    #increment RK and collect pitching totals
    while(end_of_table == False):

        #//div[3]/table/tbody/tr[1]/td
        #Gate Keeping code to determine if while loop should proceed
        try: 
            fields = [''.join(td.xpath('.//text()').extract()) for td in Selector(text=HTML).xpath('//div[3]/table/tbody/tr['+str(RK)+']/td')]
            #fields = Selector(text=HTML).xpath('//div[3]/table/tbody/tr['+str(RK)+']/td/text()').extract()
            ahrefs = Selector(text=HTML).xpath('//div[3]/table/tbody/tr['+str(RK)+']/td/a').extract()
            row_number = fields[0].encode('utf-8').strip()
            #print len(fields)
            
        except:
            try:
                #Is this because we have encoutered a "title" row? As opposed to end of table?
                fields = [''.join(td.xpath('.//text()').extract()) for td in Selector(text=HTML).xpath('//div[3]/table/tbody/tr['+str(RK+1)+']/td')]
                #fields = Selector(text=HTML).xpath('//div[3]/table/tbody/tr['+str(RK+1)+']/td/text()').extract()
                ahrefs = Selector(text=HTML).xpath('//div[3]/table/tbody/tr['+str(RK+1)+']/td/a').extract()
                row_number = fields[0].encode('utf-8').strip()

                RK += 1
            except:
                #We have reached the end of the table
                end_of_table = True
                #print year, RK, len(fields)
                

                
        #print year, row_number, len(fields), len(ahrefs)
        #Check if this year game log table contains data for DK and DF
        if len(fields) == 38:     #Table has DK, and FD information
            log_data.loc[row_number, "Year"] = year
            log_data.loc[row_number, "name"] = player
            log_data.loc[row_number, "ptype"] = "batter"
            log_data.loc[row_number, "Rk"] = row_number
            
            for i in np.arange(1, len(fields), 1):
                    
                    h = headers[i].encode('utf-8').strip()
                    log_data.loc[row_number, h] = fields[i].encode('utf-8').strip()
            
        #If no data for DK and DF, enter columns and assign null values
        elif len(fields) == 36:
            log_data.loc[row_number, "Year"] = year
            log_data.loc[row_number, "name"] = player
            log_data.loc[row_number, "ptype"] = "batter"
            log_data.loc[row_number, "RK"] = row_number
            log_data.loc[row_number, "DFS(DK)"] = ""
            log_data.loc[row_number, "DFS(FD)"] = ""
            
            for i in np.arange(1, len(fields), 1):
                    
                    h = headers[i].encode('utf-8').strip()
                    log_data.loc[row_number, h] = fields[i].encode('utf-8').strip()
        
        #From visual instection, pitcher game log tables tend to have 50 columns.  A good follow-up step would be to
        #figure this out in a more systematic way.
        elif len(fields) == 50:
            
            log_data.loc[row_number, "Year"] = year
            log_data.loc[row_number, "name"] = player
            log_data.loc[row_number, "ptype"] = "pitcher"
            log_data.loc[row_number, "Rk"] = row_number
            
            for i in np.arange(1, len(fields), 1):
                    
                    h = headers[i].encode('utf-8').strip()
                    log_data.loc[row_number, h] = fields[i].encode('utf-8').strip()
            
        else:
            exception_cases.append(RK)
            
        
        RK += 1
    
    return log_data

#get_log_data("http://www.baseball-reference.com/players/gl.cgi?id=abadfe01&t=p&year=2016", 2016)

In [16]:
def get_training_data(player):
    
    #Load the player dictionary table containing player URL extensions
    player_index = pd.read_csv("/Users/carlosbeas/sfdat26-beas/Capstone/Capstone_MLB/Data/player_index.csv")
    row_of_interest = (player_index.name == player)
    if len(row_of_interest) == 0:
        #Player was not found
        print "Error:  Player does not exist"
        empty_df = pd.DataFrame()
        return empty_df
    else:
        #Follow the URL extension found in the "links" field of the data frame
        #print player_index[row_of_interest].links
        index_of_interest = player_index[row_of_interest].index
        #print index_of_interest[0]
        extension = player_index.iloc[index_of_interest[0]]["links"]
        extension = str(extension)
        #print extension
        player_url = "http://www.baseball-reference.com"+extension
        #print player_url
        #Follow the URL and extract the additional data needed
        r = requests.get(player_url)
        #    print "got to site"
        b = BeautifulSoup(r.text, 'html.parser')
        HTML = r.text
        
        #xpath for Gamelogs:  //div[2]/div[4]/ul/li/a
        try:
            fields = Selector(text=HTML).xpath('//div[2]/div[4]/ul/li/a').extract()
            gamelogs_url = fields[2].encode('utf-8').strip()

        except:
            #fields = Selector(text=HTML).xpath('//div[5]/div[2]/div[2]/ul/li/a')
            gamelogs_url = re.search('<a href="(\/players\/gl.*)">Game Logs', HTML).group(1)
            #print gamelogs_url
        
        #years_of_interest
        years_of_interest = np.arange(2011, 2016, 1)
        years_of_interest = sorted(years_of_interest, reverse=True)
        
        player_log = pd.DataFrame()
        buffer_df = pd.DataFrame()

        for year in years_of_interest:
            
            url = "http://www.baseball-reference.com"+gamelogs_url + str(year)
            #print url
            try:

                buffer_df = get_log_data(url, year, player)
                player_log = player_log.append(buffer_df)

            except:
                #We have found a year log that does not exist for this player and it is likely that the preceeding years
                #also do not exist...
                print "ERROR:  This is not an active player."
                buffer_df = pd.DataFrame()
                break
        
    return player_log

In [17]:
#There is no data collected for DFS (DK and FD) for years 2012 and earlier.
#Creating a function that masks the dataframe to only look at data that has this information
def select_DFS_rows(df):
    
    DFS_years = (df.Year > 2012)
    DFS_df = df[DFS_years]
    
    return DFS_df

In [18]:
def calculate_rolling_means(glog):
    rol_means = pd.DataFrame()
    
    if glog.ptype[0] == "batter":
        
        #Ensure values of interest in dataframe are numeric
        #Since feature extraction analysis has not been run 
        #for pitcher, defaulting to two common features that are equally as important for pitcher
        glog.WPA = glog.WPA.map(turn_to_float)
        glog.RE24 = glog.RE24.map(turn_to_float)
        glog.RBI = glog.RBI.map(turn_to_float)
        glog.H = glog.H.map(turn_to_float)
        glog.HR = glog.HR.map(turn_to_float)
        
        #Calculate rolling mean for batter features
        RE24 = glog['RE24']
        RE24 = (RE24 - RE24.mean()) / RE24.std()
        RE24_rollmean = RE24.rolling(window=10).mean()
        rol_means["RE24"] = RE24_rollmean

        WPA = glog['WPA']
        WPA = (WPA - WPA.mean()) / WPA.std()
        WPA_rollmean = WPA.rolling(window=10).mean()
        rol_means["WPA"] = WPA_rollmean

        RBI = glog['RBI']
        RBI = (RBI - RBI.mean()) / RBI.std()
        RBI_rollmean = RBI.rolling(window=10).mean()
        rol_means["RBI"] = RBI_rollmean

        H = glog['H']
        H = (H - H.mean()) / H.std()
        H_rollmean = H.rolling(window=10).mean()
        rol_means["H"] = H_rollmean

        HR = glog['HR']
        HR = (HR - HR.mean()) / HR.std()
        HR_rollmean = HR.rolling(window=10).mean()
        rol_means["HR"] = HR_rollmean
        
        rol_means["player"] = glog.name[0]
    
    if glog.ptype[0] == "pitcher":
        
        #Ensure values of interest in dataframe are numeric
        #Since feature extraction analysis has not been run 
        #for pitcher, defaulting to two common features that are equally as important for pitcher
        glog.WPA = glog.WPA.map(turn_to_float)
        glog.RE24 = glog.RE24.map(turn_to_float)
        #Calculate rolling mean for pitcher features
        
        RE24 = glog['RE24']
        RE24 = (RE24 - RE24.mean()) / RE24.std()
        RE24_rollmean = RE24.rolling(window=10).mean()
        rol_means["RE24"] = RE24_rollmean

        WPA = glog['WPA']
        WPA = (WPA - WPA.mean()) / WPA.std()
        WPA_rollmean = WPA.rolling(window=10).mean()
        rol_means["WPA"] = WPA_rollmean
        
        rol_means["player"] = glog.name[0]

    
    return rol_means

def get_rolling_mean(player):
    
    today = dt.datetime.today().strftime("%m/%d/%Y")
    today = today.split("/")
    this_month = today[0]
    this_day = today[1]
    this_year = today[2]
    
    
    #Load the player dictionary table containing player URL extensions
    player_index = pd.read_csv("/Users/carlosbeas/sfdat26-beas/Capstone/Capstone_MLB/Data/player_index.csv")
    row_of_interest = (player_index.name == player)
    if len(row_of_interest) == 0:
        #Player was not found
        print "Error:  Player does not exist"
        empty_df = pd.DataFrame()
        return empty_df
    else:
        #Follow the URL extension found in the "links" field of the data frame
        #print player_index[row_of_interest].links
        index_of_interest = player_index[row_of_interest].index
        #print index_of_interest[0]
        extension = player_index.iloc[index_of_interest[0]]["links"]
        extension = str(extension)
        #print extension
        player_url = "http://www.baseball-reference.com"+extension
        #print player_url
        #Follow the URL and extract the additional data needed
        r = requests.get(player_url)
        #    print "got to site"
        b = BeautifulSoup(r.text, 'html.parser')
        HTML = r.text
        
        #xpath for Gamelogs:  //div[2]/div[4]/ul/li/a
        try:
            fields = Selector(text=HTML).xpath('//div[2]/div[4]/ul/li/a').extract()
            gamelogs_url = fields[2].encode('utf-8').strip()

        except:
            #fields = Selector(text=HTML).xpath('//div[5]/div[2]/div[2]/ul/li/a')
            gamelogs_url = re.search('<a href="(\/players\/gl.*)">Game Logs', HTML).group(1)
            #print gamelogs_url
            
        url = "http://www.baseball-reference.com"+gamelogs_url + str(this_year)
        
        #print url
        try:

            game_log = get_log_data(url, this_year, player)

        except:
            #We have found a year log that does not exist for this player and it is likely that the preceeding years
            #also do not exist...
            print "ERROR:  This is not an active player."
            empty_df = pd.DataFrame()
            return empty_df
        
        #print game_log.shape
        
        my_rolling_mean = pd.DataFrame()
        months = []
        days = []
    
        #gameday = game_log.Date.map(lambda x: x.split())
        for i in np.arange(0, game_log.shape[0], 1):
            gameday = game_log.Date[i]
            gameday = str(gameday)
            month = re.search('(\w+)', gameday).group(1)
            #print month
            day = re.search('(\d+)', gameday).group(1)
            #print day
            months.append(month)
            days.append(day)
    #       print day
            
        #print months
        game_log["toDate"] = game_log["Year"] + "-" + months + "-" + days
        #print game_log["toDate"]
        game_log.toDate = pd.to_datetime(game_log.toDate)
        game_log.set_index('toDate', inplace=True)
        
        #get rolling mean of features for player
        my_rolling_mean = calculate_rolling_means(game_log)
        my_rolling_mean = my_rolling_mean.iloc[-1]
        
    return my_rolling_mean

In [19]:
#Load the player dictionary table containing player URL extensions
player_index = pd.read_csv("/Users/carlosbeas/sfdat26-beas/Capstone/Capstone_MLB/Data/player_index.csv")

player_dataframe = pd.DataFrame()
names = player_index.name.unique()


##Extract scores table and
##Append it to the "Master DataFrame" (which cointains the the information for all players combined)
for name in names:
    player_dataframe = player_dataframe.append(get_training_data(name))

# def master_DF_ETL():
    
#     #Load the player dictionary table containing player URL extensions
#     player_index = pd.read_csv("/Users/carlosbeas/sfdat26-beas/Capstone/Capstone_MLB/Data/player_index.csv")
    
#     print player_index
#     #For every player found in the index table:
    
#     #for indx, player in enumerate player_index:
#     #    print indx, player
    
#     #Extract scores table and
#     #Append it to the "Master DataFrame" (which cointains the the information for all players combined)
    
#     return 0

#master_DF_ETL

AttributeError: 'NoneType' object has no attribute 'group'

In [12]:
Brandon_Crawford = get_training_data("Brandon Crawford")

In [13]:
Joey_Votto = get_training_data("Joey Votto")

In [14]:
Jacoby_Ellsbury = get_training_data("Jacoby Ellsbury")

In [16]:
David_Wright = get_training_data("David Wright")

In [17]:
Mike_Trout = get_training_data("Mike Trout")

In [18]:
Alex_Rodriguez = get_training_data("Alex Rodriguez")

In [35]:
#Create a "Master table" that contains the info for all players 
master_df = Brandon_Crawford.append(Joey_Votto)
master_df = master_df.append(Jacoby_Ellsbury)
master_df = master_df.append(David_Wright)
master_df = master_df.append(Mike_Trout)
master_df = master_df.append(Alex_Rodriguez)

In [36]:
#Limit the table to only years of interest
master_df = select_DFS_rows(master_df)

#Identify the players stored in the dataframe
my_players = master_df.name.unique()

In [90]:
master_df = format_batter_numeric_rows(master_df)
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2173 entries, 1 to 44
Data columns (total 42 columns):
            2173 non-null object
doubles     2173 non-null float64
tripples    2173 non-null float64
AB          2173 non-null float64
BA          2173 non-null float64
BB          2173 non-null float64
BOP         2173 non-null float64
CS          2173 non-null float64
DK          2173 non-null float64
FD          2173 non-null float64
Date        2173 non-null object
GDP         2173 non-null object
Gcar        2173 non-null float64
Gtm         2173 non-null object
H           2173 non-null float64
HBP         2173 non-null float64
HR          2173 non-null float64
IBB         2173 non-null object
Inngs       2173 non-null object
OBP         2173 non-null float64
OPS         2173 non-null float64
Opp         2173 non-null object
PA          2173 non-null float64
Pos         2173 non-null object
R           2173 non-null float64
RBI         2173 non-null float64
RE24        2173 non-nul

,,doubles,tripples,AB,BA,BB,BOP,CS,DK,FD,...,SF,SH,SLG,SO,Tm,WPA,Year,aLI,name,ptype
1,@,0.0,0.0,4.0,0.500,0.0,7.0,0.0,8.0,2.50,...,0,0.0,0.500,0.0,SFG,0.163,2015.0,.92,Brandon Crawford,batter
2,@,0.0,0.0,4.0,0.375,0.0,7.0,0.0,14.0,5.25,...,0,0.0,0.750,1.0,SFG,-0.033,2015.0,1.30,Brandon Crawford,batter
3,@,0.0,0.0,5.0,0.385,0.0,7.0,0.0,10.0,3.25,...,0,0.0,0.615,0.0,SFG,0.066,2015.0,.78,Brandon Crawford,batter
4,@,0.0,0.0,5.0,0.278,0.0,5.0,0.0,2.0,-0.25,...,0,0.0,0.444,1.0,SFG,-0.078,2015.0,1.40,Brandon Crawford,batter
5,@,0.0,0.0,3.0,0.238,0.0,5.0,0.0,0.0,-0.75,...,0,0.0,0.381,2.0,SFG,-0.227,2015.0,2.37,Brandon Crawford,batter


In [41]:
master_df.drop("RK", inplace=True)

In [42]:
rename_map = {
    # Original column: [renamed column]
    '2B': 'doubles',
    '3B': 'tripples',
    'DFS(DK)': 'DK',
    'DFS(FD)': 'FD'
}

master_df.rename(columns=rename_map, inplace=True)

In [51]:
#Create a random forest regressor model for Draft Kings Score for each player

rfr = []
for indx, player in enumerate(my_players):
    
    rfr_buff = RandomForestRegressor(max_depth=5, max_leaf_nodes=7, min_samples_split=4, n_estimators=150, random_state=1, n_jobs=1)
    rfr.append(rfr_buff)

    player_mask = (master_df.name == player)
    player_df = master_df[player_mask]
    
    #Define my target and features
    y, X = patsy.dmatrices('DK ~ RE24+WPA+H+HR+RBI', data=player_df, return_type="dataframe")
    y = np.ravel(y)
    
    rfr[indx].fit(X, y)
    

In [73]:
#Extract information and calculate the rolling mean per player
#Brandon_Crawford_features = get_rolling_mean("Brandon Crawford")

#todays_features = pd.DataFrame()
# 
player_features = get_rolling_mean(my_players[0])
todays_features = pd.DataFrame(player_features)
todays_features = todays_features.T

for indx, player in enumerate(my_players):
    if indx > 0:
        player_features = get_rolling_mean(player)
        player_features = pd.DataFrame(player_features)
        player_features = player_features.T
        todays_features = todays_features.append(player_features)


todays_features

,RE24,WPA,RBI,H,HR,player
2016-10-02,0.299603,0.139289,-0.142642,0.410211,-0.288747,Brandon Crawford
2016-10-02,0.393032,0.200931,0.403899,0.462374,0.535235,Joey Votto
2016-10-02,0.222553,0.0995128,0.0295294,-0.0832166,-0.253596,Jacoby Ellsbury
2016-05-27,0.00899823,0.104525,0.307476,-0.0495095,0.23998,David Wright
2016-10-02,-0.0956935,0.0731786,-0.131839,-0.300927,-0.212682,Mike Trout
2016-08-12,-0.185679,-0.216583,-0.368131,-0.594786,-0.397796,Alex Rodriguez


In [84]:
features = X.columns
X_pred = pd.DataFrame(columns=features)
my_Draft_kings_Scores = pd.DataFrame(columns=("player", "score"))

player_mask = (todays_features.player == "Brandon Crawford")
X_pred = todays_features[player_mask]
features

Index([u'Intercept', u'RE24', u'WPA', u'H', u'HR', u'RBI'], dtype='object')

In [89]:
#Use the fit models and extracted features to make Draft Kings prediction

features = X.columns
X_pred = pd.DataFrame(columns=features)
my_Draft_kings_Scores = {"player":[],
                         "score":[]
                        }


for indx, player in enumerate(my_players):
    
    player_mask = (todays_features.player == player)
    X_pred = todays_features[player_mask]
    X_pred.drop("player", axis=1, inplace=True)
    X_pred["Intercept"] = 1
    player_prediction = rfr[indx].predict(X_pred)
    my_Draft_kings_Scores["player"].append(player)
    my_Draft_kings_Scores["score"].append(player_prediction)
    
my_Draft_kings_Scores = pd.DataFrame(my_Draft_kings_Scores)
my_Draft_kings_Scores

/Users/carlosbeas/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/carlosbeas/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,player,score
0,Brandon Crawford,[1.91688197417]
1,Joey Votto,[13.1605672066]
2,Jacoby Ellsbury,[4.16500266581]
3,David Wright,[5.07137299276]
4,Mike Trout,[4.40630951121]
5,Alex Rodriguez,[4.29119453044]
